In [ ]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
from sklearn import tree
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *
'''
    ISOT
'''


read_file = r"../data/Des_fake_news/LIAR.xlsx"
read_sheet = "valid"
write_file = r"../data/Des_fake_news/LIAR_PROCESSED_valid.csv"

data = pd.read_excel(read_file, read_sheet)

data = data.dropna(axis=0)
data

,id,class,text,title,speaker,job title,state,party,barelytrues,falses,halftrues,mostlystrues,pantsonfires,context,flag
0,11972.json,True,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30.0,30.0,42.0,23.0,18.0,Radio interview,real
1,11685.json,False,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2.0,1.0,0.0,0.0,0.0,a news conference,fake
2,11096.json,False,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63.0,114.0,51.0,37.0,61.0,comments on ABC's This Week.,fake
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1.0,1.0,3.0,1.0,1.0,a radio show,fake
6,7070.json,True,Says that Tennessee law requires that schools ...,"county-budget,county-government,education,taxes",stand-children-tennessee,Child and education advocacy organization.,Tennessee,none,0.0,0.0,0.0,0.0,0.0,in a post on Facebook.,real
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,9436.json,barely-true,We came out of the White House not only dead b...,"candidates-biography,elections",hillary-clinton,Presidential candidate,New York,democrat,40.0,29.0,69.0,76.0,7.0,an interview on ABC,fake
1278,7334.json,half-true,Says his budget provides the highest state fun...,education,rick-scott,Governor,Florida,republican,28.0,23.0,38.0,34.0,7.0,a news conference,fake
1279,9788.json,barely-true,Ive been here almost every day.,"civil-rights,crime,criminal-justice",jay-nixon,Governor,Missouri,democrat,2.0,0.0,0.0,1.0,0.0,"on ABC's ""This Week""",fake
1280,10710.json,barely-true,"In the early 1980s, Sen. Edward Kennedy secret...","bipartisanship,congress,foreign-policy,history",mackubin-thomas-owens,"senior fellow, Foreign Policy Research Institute",Rhode Island,columnist,1.0,0.0,0.0,0.0,0.0,a commentary in The Providence Journal,fake


In [20]:
#remove nonASCIICharacters

data["title"] = data.apply(lambda x: "".join([i for i in x["title"] if ord(i) <= 127]), axis=1)



data["text"] = data.apply(lambda x: "".join([i for i in x["text"] if ord(i) <= 127]), axis=1)


In [21]:
def to_class(s):
    match s:
        case "real":
            return 1
        case "fake":
            return 0

In [22]:
data["flag"] = data.apply(lambda x: to_class(x["flag"]), axis=1)
data

,id,class,text,title,speaker,job title,state,party,barelytrues,falses,halftrues,mostlystrues,pantsonfires,context,flag
0,11972.json,True,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30.0,30.0,42.0,23.0,18.0,Radio interview,1
1,11685.json,False,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2.0,1.0,0.0,0.0,0.0,a news conference,0
2,11096.json,False,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63.0,114.0,51.0,37.0,61.0,comments on ABC's This Week.,0
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1.0,1.0,3.0,1.0,1.0,a radio show,0
6,7070.json,True,Says that Tennessee law requires that schools ...,"county-budget,county-government,education,taxes",stand-children-tennessee,Child and education advocacy organization.,Tennessee,none,0.0,0.0,0.0,0.0,0.0,in a post on Facebook.,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,9436.json,barely-true,We came out of the White House not only dead b...,"candidates-biography,elections",hillary-clinton,Presidential candidate,New York,democrat,40.0,29.0,69.0,76.0,7.0,an interview on ABC,0
1278,7334.json,half-true,Says his budget provides the highest state fun...,education,rick-scott,Governor,Florida,republican,28.0,23.0,38.0,34.0,7.0,a news conference,0
1279,9788.json,barely-true,Ive been here almost every day.,"civil-rights,crime,criminal-justice",jay-nixon,Governor,Missouri,democrat,2.0,0.0,0.0,1.0,0.0,"on ABC's ""This Week""",0
1280,10710.json,barely-true,"In the early 1980s, Sen. Edward Kennedy secret...","bipartisanship,congress,foreign-policy,history",mackubin-thomas-owens,"senior fellow, Foreign Policy Research Institute",Rhode Island,columnist,1.0,0.0,0.0,0.0,0.0,a commentary in The Providence Journal,0


In [23]:
data["text_tb_pol"] = data.apply(lambda x: TextBlob(x["text"]).polarity, axis=1)
data["text_tb_sub"] = data.apply(lambda x: TextBlob(x["text"]).subjectivity, axis=1)

data["title_tb_pol"] = data.apply(lambda x: TextBlob(x["title"]).polarity, axis=1)
data["title_tb_sub"] = data.apply(lambda x: TextBlob(x["title"]).subjectivity, axis=1)
data

,id,class,text,title,speaker,job title,state,party,barelytrues,falses,halftrues,mostlystrues,pantsonfires,context,flag,text_tb_pol,text_tb_sub,title_tb_pol,title_tb_sub
0,11972.json,True,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30.0,30.0,42.0,23.0,18.0,Radio interview,1,0.000000,0.000000,0.0,0.0
1,11685.json,False,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2.0,1.0,0.0,0.0,0.0,a news conference,0,0.000000,0.000000,0.0,0.0
2,11096.json,False,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63.0,114.0,51.0,37.0,61.0,comments on ABC's This Week.,0,0.000000,0.000000,0.0,0.0
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1.0,1.0,3.0,1.0,1.0,a radio show,0,0.000000,0.000000,0.0,0.0
6,7070.json,True,Says that Tennessee law requires that schools ...,"county-budget,county-government,education,taxes",stand-children-tennessee,Child and education advocacy organization.,Tennessee,none,0.0,0.0,0.0,0.0,0.0,in a post on Facebook.,1,-0.166667,0.166667,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,9436.json,barely-true,We came out of the White House not only dead b...,"candidates-biography,elections",hillary-clinton,Presidential candidate,New York,democrat,40.0,29.0,69.0,76.0,7.0,an interview on ABC,0,-0.066667,0.466667,0.0,0.0
1278,7334.json,half-true,Says his budget provides the highest state fun...,education,rick-scott,Governor,Florida,republican,28.0,23.0,38.0,34.0,7.0,a news conference,0,0.000000,0.000000,0.0,0.0
1279,9788.json,barely-true,Ive been here almost every day.,"civil-rights,crime,criminal-justice",jay-nixon,Governor,Missouri,democrat,2.0,0.0,0.0,1.0,0.0,"on ABC's ""This Week""",0,0.000000,0.000000,0.0,0.0
1280,10710.json,barely-true,"In the early 1980s, Sen. Edward Kennedy secret...","bipartisanship,congress,foreign-policy,history",mackubin-thomas-owens,"senior fellow, Foreign Policy Research Institute",Rhode Island,columnist,1.0,0.0,0.0,0.0,0.0,a commentary in The Providence Journal,0,-0.150000,0.500000,0.0,0.0


In [24]:
analyzer = SentimentIntensityAnalyzer()

data["title_vader_scores"] = data.apply(lambda x: analyzer.polarity_scores(x["title"]), axis=1)
data["title_vader_comp"] = data.apply(lambda x: x["title_vader_scores"]["compound"], axis=1)
data["title_vader_neg"] = data.apply(lambda x: x["title_vader_scores"]["neg"], axis=1)
data["title_vader_neu"] = data.apply(lambda x: x["title_vader_scores"]["neu"], axis=1)
data["title_vader_pos"] = data.apply(lambda x: x["title_vader_scores"]["pos"], axis=1)
data = data.drop(["title_vader_scores"], axis=1)

data["text_vader_scores"] = data.apply(lambda x: analyzer.polarity_scores(x["text"]), axis=1)
data["text_vader_comp"] = data.apply(lambda x: x["text_vader_scores"]["compound"], axis=1)
data["text_vader_neg"] = data.apply(lambda x: x["text_vader_scores"]["neg"], axis=1)
data["text_vader_neu"] = data.apply(lambda x: x["text_vader_scores"]["neu"], axis=1)
data["text_vader_pos"] = data.apply(lambda x: x["text_vader_scores"]["pos"], axis=1)
data = data.drop(["text_vader_scores"], axis=1)

In [25]:
def discrete_vader(neg, neu, pos):
    return np.argmax([neg, neu, pos])
    
def discrete_textblob(pol):
    if pol <= -0.33:
        return 0
    elif pol > -0.33 and pol < 0.33:
        return 1
    else:
        return 2

def discrete_textblob_sub(pol):
    if pol < 0.5:
        return 0
    return 1

In [26]:

data["title_vader_class"] = data.apply(lambda x: discrete_vader(x["title_vader_neg"], x["title_vader_neu"], x["title_vader_pos"]), axis=1)
data["text_vader_class"] = data.apply(lambda x: discrete_vader(x["text_vader_neg"], x["text_vader_neu"], x["text_vader_pos"]), axis=1)

data["text_tb_pol_class"] = data.apply(lambda x: discrete_textblob(x["text_tb_pol"]), axis=1)
data["text_tb_sub_class"] = data.apply(lambda x: discrete_textblob_sub(x["text_tb_sub"]), axis=1)

data["title_tb_pol_class"] = data.apply(lambda x: discrete_textblob(x["title_tb_pol"]), axis=1)
data["title_tb_sub_class"] = data.apply(lambda x: discrete_textblob_sub(x["title_tb_sub"]), axis=1)
data


,id,class,text,title,speaker,job title,state,party,barelytrues,falses,...,text_vader_comp,text_vader_neg,text_vader_neu,text_vader_pos,title_vader_class,text_vader_class,text_tb_pol_class,text_tb_sub_class,title_tb_pol_class,title_tb_sub_class
0,11972.json,True,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30.0,30.0,...,0.0000,0.000,1.000,0.000,1,1,1,0,1,0
1,11685.json,False,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2.0,1.0,...,0.0772,0.000,0.894,0.106,1,1,1,0,1,0
2,11096.json,False,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63.0,114.0,...,-0.3089,0.201,0.799,0.000,1,1,1,0,1,0
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1.0,1.0,...,0.3400,0.127,0.602,0.271,1,1,1,0,1,0
6,7070.json,True,Says that Tennessee law requires that schools ...,"county-budget,county-government,education,taxes",stand-children-tennessee,Child and education advocacy organization.,Tennessee,none,0.0,0.0,...,0.3182,0.000,0.919,0.081,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,9436.json,barely-true,We came out of the White House not only dead b...,"candidates-biography,elections",hillary-clinton,Presidential candidate,New York,democrat,40.0,29.0,...,-0.0933,0.179,0.606,0.214,1,1,1,0,1,0
1278,7334.json,half-true,Says his budget provides the highest state fun...,education,rick-scott,Governor,Florida,republican,28.0,23.0,...,0.0000,0.000,1.000,0.000,1,1,1,0,1,0
1279,9788.json,barely-true,Ive been here almost every day.,"civil-rights,crime,criminal-justice",jay-nixon,Governor,Missouri,democrat,2.0,0.0,...,0.0000,0.000,1.000,0.000,1,1,1,0,1,0
1280,10710.json,barely-true,"In the early 1980s, Sen. Edward Kennedy secret...","bipartisanship,congress,foreign-policy,history",mackubin-thomas-owens,"senior fellow, Foreign Policy Research Institute",Rhode Island,columnist,1.0,0.0,...,0.4019,0.000,0.881,0.119,1,1,1,1,1,0


In [27]:
#data.to_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv", index=False)

In [28]:
'''
    Begin Neural Net for news text
'''

#data = pd.read_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv")
'''text only'''
features = data[["text_tb_pol", "text_vader_pos", "text_vader_neg", "text_vader_neu"]].to_numpy()


import tensorflow as tf
from sklearn.model_selection import train_test_split

imdb_model = tf.keras.models.load_model(r"SA_imdb/IMDB_NN.keras")
imdb_sentiment = np.argmax(imdb_model.predict(features), axis=-1)

tweets_model = tf.keras.models.load_model(r"SA_tweets/TWEETS_NN.keras")
tweets_sentiment = np.argmax(tweets_model.predict(features), axis=-1)

data["text_NN_imdb"] = imdb_sentiment
data["text_NN_tweets"] = tweets_sentiment

#data.to_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv", index=False)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [29]:
'''
    Begin Neural Net for news title
'''


'''text only'''
features = data[["title_tb_pol", "title_vader_pos", "title_vader_neg", "title_vader_neu"]].to_numpy()


import tensorflow as tf
from sklearn.model_selection import train_test_split

imdb_model_two = tf.keras.models.load_model(r"SA_imdb/IMDB_NN.keras")
imdb_sentiment_two = np.argmax(imdb_model_two.predict(features), axis=-1)

tweets_model_two = tf.keras.models.load_model(r"SA_tweets/TWEETS_NN.keras")
tweets_sentiment_two = np.argmax(tweets_model_two.predict(features), axis=-1)

data["title_NN_imdb"] = imdb_sentiment_two
data["title_NN_tweets"] = tweets_sentiment_two

#data.to_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv", index=False)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [30]:
'''
    Begin log regression for text
'''
from sklearn.linear_model import LogisticRegression
import pickle

texts = data[["text_tb_pol", "text_vader_pos", "text_vader_neg", "text_vader_neu"]].to_numpy()

tweet_text_log:  LogisticRegression = None
with open(r"SA_tweets/TWEETS_LOG.pkl", "rb") as file:
    tweet_text_log = pickle.load(file)

imdb_text_log:  LogisticRegression = None
with open(r"SA_imdb/IMDB_LOG.pkl", "rb") as file:
    imdb_text_log = pickle.load(file)

data["text_log_imdb"] = imdb_text_log.predict(texts)
data["text_log_tweets"] = imdb_text_log.predict(texts)





In [31]:
'''
    Begin log regression for title
'''
from sklearn.linear_model import LogisticRegression
import pickle

titles = data[["title_tb_pol", "title_vader_pos", "title_vader_neg", "title_vader_neu"]].to_numpy()


tweet_text_log:  LogisticRegression = None
with open(r"SA_tweets/TWEETS_LOG.pkl", "rb") as file:
    tweet_text_log = pickle.load(file)

imdb_text_log:  LogisticRegression = None
with open(r"SA_imdb/IMDB_LOG.pkl", "rb") as file:
    imdb_text_log = pickle.load(file)

data["title_log_imdb"] = imdb_text_log.predict(texts)
data["title_log_tweets"] = imdb_text_log.predict(texts)


data.to_csv(write_file, index=False)